Launch the following blocks to connect to your drive and go into the tutorial folder

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd /content/gdrive/My\ Drive/TD_Dreem_MasterBin/Dreem_Master_Bin
! ls

This tutorial is about machine learning methods for sleep stage classification.


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

from sklearn.metrics import balanced_accuracy_score, cohen_kappa_score, confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from dreem_master_bin.hypnogram import plot_hypnogram, stage_colors

To save you some time, train and test datasets are already available in the data folder. It consists of preprocessed data, not raw record. You have two types of datasets:

- Spectral dataset = data containing spectral power (spectrogram matrix): train and test
- Features dataset = data containing precomputed features: train and test

As with real-life data, we will not be allowed to use test data for training. The test data will only be used for the final evaluation.

In [5]:
from dreem_master_bin.load_data import load_spectral_datasets, load_feature_datasets

# load spectrogram dataset and shuffle train data
x_train_spect, y_train_spect, x_test_spect, y_test_spect = load_spectral_datasets()
spectral_names = ['index_window', '1Hz', '2Hz', '3Hz', '4Hz', '5Hz', '6Hz', '7Hz',
                  '8Hz', '9Hz', '10Hz', '11Hz', '12Hz', '13Hz', '14Hz', '15Hz', 
                  '16Hz', '18Hz', '19Hz']
# shuffle train dataset
p = np.random.permutation(len(y_train_spect))
x_train_spect, y_train_spect = x_train_spect[p], y_train_spect[p]


# load precomputed features dataset and shuffle train data
x_train_feat, y_train_feat, x_test_feat, y_test_feat = load_feature_datasets()
features_name = ['index_window', 'delta', 'delta_r', 'theta', 'theta_r',
       'lowfreq', 'lowfreq_r', 'alpha', 'alpha_r', 'sigma', 'sigma_r', 'beta',
       'beta_r', 'kcomp', 'kcomp_r', 'SC', 'SEF90', 'SEF95', 'Nb spindles',
       'spindles magnitude', 'spindles duration', 'Nb slow waves',
       'slow waves magnitude', 'slow waves duration', 'AccelerometerVar',
       'little movement', 'strong movement']
# shuffle train dataset
p = np.random.permutation(len(y_train_feat))
x_train_feat, y_train_feat = x_train_feat[p], y_train_feat[p]


We have just loaded the spectral data:

- x_train_spect, x_test_spect: spectral data to predict sleep stages
It is an array of shape n_samples x n_features

    - n_samples = number of sleep epochs
    - n_features = number of features for each of these epochs. The features are: [index_window, power_frequency_1Hz, power_frequency_2Hz, ..., power_frequency_18Hz], where index window to the position of the sample in its sleep record.
- y_train_spect, y_test_spect: labels (sleep stages)

Then, we have loaded the other dataset:

- x_train_spect, x_test_spect: shape n_samples x n_features
    - n_features = the features are ['index_window', 'delta', 'delta_r', 'theta', 'theta_r', 'lowfreq', 'lowfreq_r', 'alpha', 'alpha_r', 'sigma', 'sigma_r', 'beta', 'beta_r', 'kcomp', 'kcomp_r', 'SC', 'SEF90', 'SEF95', 'Nb spindles', 'spindles magnitude', 'spindles duration', 'Nb slow waves', 'slow waves magnitude', 'slow waves duration', 'AccelerometerVar','little movement', 'strong movement']

- y_train_spect, y_test_spect: labels (sleep stages)



We need also to define a function to will evaluate the models

In [6]:
def evaluate(model, x_data, true_labels):
    predictions = model.predict(x_data)
    scores = {'balanced_accuracy': balanced_accuracy_score(true_labels, predictions),
            'cohen_kappa': cohen_kappa_score(true_labels, predictions),
            'confusion_matrix': confusion_matrix(true_labels, predictions)}

    return scores

Let's start with the spectral dataset !

In [7]:
x_train, y_train = x_train_spect, y_train_spect
x_test, y_test = x_test_spect, y_test_spect

1 - Feature scaling + Dimension reduction + classifier

It is crucial to process your raw data before inserting them in a classifier. 
In this section, you will work with data that have already been processed (spectral and features dataset). Then, you can:
- scale your data (here we are using StandardScaler)
- apply a dimension reduction (PCA, ICA...)

After that you can choose your classifier (e.g SVM classifier)


*You can go to the online documentation of the scikit-library to find similar functions, with the keywords:* 
- *multi class classifier*
- *dimension reduction*
- *decomposition*


In [ ]:
from sklearn.decomposition import PCA
from sklearn.svm import SVC

# scale input data and reduce dimension
pca = make_pipeline(StandardScaler(),
                    PCA(n_components=5, random_state=10))
pca.fit(x_train, y_train)

# linear classifier
classifier = SVC(kernel='linear')
# training: fit the model to the data
print('training')
classifier.fit(pca.transform(x_train), y_train)

# evaluation
evaluate(classifier, pca.transform(x_test), y_test)

You can see that we predict almost no N1 stage: this may be due to the class distribution inside the dataset. Let's have a look on the train and test datasets:

In [ ]:
# percentage of sleep stages in y_train and y_test
stage_correspondance = {0: "WAKE", 1: "N1", 2: "N2", 3: "DEEP", 4: "REM"}

print('train dataset')
for k, stage_name in stage_correspondance.items():
    percentage = round(100 * sum(y_train==k) / len(y_train), 2)
    print(stage_name + ': ' + str(percentage) + '%')

print('')
print('test dataset')
for k, stage_name in stage_correspondance.items():
    percentage = round(100 * sum(y_test==k) / len(y_test), 2)
    print(stage_name + ': ' + str(percentage) + '%')

You can notice that the datasets are unbalanced. Indeed, as they were made from whole sleep records, the distribution of sleep stages in the dataset is characteristic of the average percentage of each sleep stage during a night sleep.

2 - Tune your model


Here we are going to use the Random Forest classifier, one of the many ensemble learning functions of the scikit-learn library: https://scikit-learn.org/stable/modules/ensemble.html

Also, we are going to work with the preprocessed features dataset. 


In [ ]:
# load features dataset and shuffle train data
x_train, y_train = x_train_feat, y_train_feat
x_test, y_test = x_test_feat, y_test_feat

# select a classifier and train it
from sklearn.ensemble import RandomForestClassifier

rf = make_pipeline(StandardScaler(),
                       RandomForestClassifier())
print('training...')
rf.fit(x_train, y_train)

# evaluation
evaluate(rf, x_test, y_test)

If you look at the documentation of RandomForestClassifier, you will see that it has many arguments. Most of them are hyper-parameters. The training process will only change the parameters of the model, but it is up to you to find the best hyper-parameters.

In [ ]:
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params()['randomforestclassifier'])

We will try adjusting the following set of hyperparameters:

- n_estimators = number of trees in the foreset
- max_features = max number of features considered for splitting a node
- max_depth = max number of levels in each decision tree
- min_samples_split = min number of data points placed in a node before the node is split
- min_samples_leaf = min number of data points allowed in a leaf node
- bootstrap = method for sampling data points (with or without replacement)

To do this, we are going to perform a randomized search (with cross-validation) using RandomizedSearchCV. RandomizedSearchCV peerforms a randomized search in the hyperparameter space to find the best model, it goes then faster than a complete search over all the combinations of parameters.

The number of fit permormed is equal to cv * n_iter, each fit takes between 1 and 3 minutes.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 800, stop = 1600, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(20, 30, num = 3)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)


# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=rf, 
                               param_distributions=random_grid, 
                               n_iter=3, # Number of parameter settings that are sampled. n_iter trades off runtime vs quality of the solution.
                               cv=3, # Determines the cross-validation splitting strategy
                               verbose=2, 
                               random_state=42, 
                               n_jobs = 8)
# Fit the random search model
rf_random.fit(x_train, y_train)



Have a look at this article during the fit: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74 

It explains the concept behind RandomizedSearchCV:
- randomized search over a space of hyperparameters
- Cross-validation: if you only work with a training dataset, you increase your chance of overfitting.

In [ ]:
# look at the best params computed from RandomizedSearchCV
print('Best parameters from RandomizedSearCV')
pprint(rf_random.best_params_)

print('')
print('Base model: default value for hyperparameters')
base_model = rf.fit(x_train, y_train)
base_accuracy = evaluate(base_model, x_test, y_test)
pprint(base_accuracy)

print('')
print('Best model from RandomizedSearCV')
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, x_test, y_test)
pprint(random_accuracy)


3 - Stack multiple estimators

It is possible to combine multiple machine learning algorithms to improve performance.

> Use StackingClassifier to stack estimators with a final classifier


In [ ]:
# select a classifier and train it
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

rf_pipeline = make_pipeline(StandardScaler(),
                            RandomForestClassifier(n_estimators=10, random_state=42))
gradient_pipeline = make_pipeline(StandardScaler(),
                                  HistGradientBoostingClassifier(learning_rate=0.01, random_state=30))
estimators = [('Random Forest', rf_pipeline),
                  ('Gradient Boosting', gradient_pipeline)]
stacking_classifier = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(max_iter=200))

print('training...')
stacking_classifier.fit(x_train, y_train)

# evaluation
evaluate(rf, x_test, y_test)

4 - Feature importance

In all this tutorial, we have tried to predict sleep stages from precomputed features (spectral or other features). 

Scikit provides methods to assess to importance of each feature for the prediction.

In [ ]:
# load spectrogram dataset and shuffle train data
x_train, y_train = x_train_feat, y_train_feat
list_features = features_name

# let's take an already trained classifier
clf = stacking_classifier

# permutation importance > feature importance
print('permutations...')
from sklearn.inspection import permutation_importance
result = permutation_importance(clf, x_train, y_train, n_repeats=10, random_state=0)

# sort by importance
sorted_idx = result.importances_mean.argsort()

# Plot
fig, ax = plt.subplots(figsize=(25, 10))
ax.boxplot(result.importances[sorted_idx].T,
           vert=False, labels=[list_features[i] for i in sorted_idx])
ax.set_title("Permutation Importances (train set)")
fig.tight_layout()
plt.show()


You've reached the end of this second tutorial.

Let's go to the last part about deep learning methods.
Open the **Tutorial_Sleep_Staging_C** tutorial